Walidacja wybranych modeli

In [4]:
from pickle import FALSE
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import time
import streamlit as st
import warnings 
import requests
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

In [117]:
dataset = pd.read_csv("daneSkinCare.csv", sep=';')
dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]

In [118]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                154 non-null    int64 
 1   Typ cery          154 non-null    object
 2   Wrażliwa          154 non-null    int64 
 3   Główny problem    154 non-null    object
 4   Poboczny problem  154 non-null    object
 5   Wiek              154 non-null    int64 
 6   Mycie             154 non-null    object
 7   Serum na dzień    154 non-null    object
 8   Krem na dzień     154 non-null    object
 9   SPF               154 non-null    object
 10  Serum na noc      154 non-null    object
 11  Krem na noc       154 non-null    object
 12  Punktowo          154 non-null    object
 13  Maseczka          154 non-null    object
 14  Peeling           154 non-null    object
dtypes: int64(3), object(12)
memory usage: 18.2+ KB


In [119]:
def clearSkinTypeMisspelledData(datasetToClean):
    datasetToClean.loc[(datasetToClean["Typ cery"] == "tłusta") | 
                        (datasetToClean["Typ cery"] == "tlusta") | 
                        (datasetToClean["Typ cery"] == "Tlusta"), "Typ cery"] = "Tłusta"
    datasetToClean.loc[(datasetToClean["Typ cery"] == "mieszana"), "Typ cery"] = "Mieszana"
    datasetToClean.loc[(datasetToClean["Typ cery"] == "sucha"), "Typ cery"] = "Sucha"
    datasetToClean.loc[(datasetToClean["Typ cery"] == "normalna"), "Typ cery"] = "Normalna"

def clearMainOrsecondProblemMisspelledData(datasetToClean, mainOrsecondProblem):
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "nadprodukcja sebum") | 
                        (datasetToClean[mainOrsecondProblem] == "nadprodukcja Sebum"), mainOrsecondProblem] = "Nadprodukcja sebum"
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "niedoskonałości") | 
                        (datasetToClean[mainOrsecondProblem] == "niedoskonalosci") |
                        (datasetToClean[mainOrsecondProblem] == "niedoskonałosci") |
                        (datasetToClean[mainOrsecondProblem] == "niedoskonalości") |
                        (datasetToClean[mainOrsecondProblem] == "Niedoskonalości") | 
                        (datasetToClean[mainOrsecondProblem] == "Niedoskonalosci") |
                        (datasetToClean[mainOrsecondProblem] == "Niedoskonałosci"), mainOrsecondProblem] = "Niedoskonałości"
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "podrażnienie") | 
                        (datasetToClean[mainOrsecondProblem] == "podraznienie") |
                        (datasetToClean[mainOrsecondProblem] == "Podraznienie") , mainOrsecondProblem] = "Podrażnienie"
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "przebarwienia"), mainOrsecondProblem] = "Przebarwienia"
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "rozszerzone pory") | 
                        (datasetToClean[mainOrsecondProblem] == "Rozszerzone Pory") |
                        (datasetToClean[mainOrsecondProblem] == "rozszerzone Pory") , mainOrsecondProblem] = "Rozszerzone pory"
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "suche skórki") | 
                        (datasetToClean[mainOrsecondProblem] == "suche Skórki") |
                        (datasetToClean[mainOrsecondProblem] == "Suche skorki") |
                        (datasetToClean[mainOrsecondProblem] == "suche skorki") |
                        (datasetToClean[mainOrsecondProblem] == "suche Skorki"), mainOrsecondProblem] = "Suche skórki"
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "szara cera") | 
                        (datasetToClean[mainOrsecondProblem] == "szara Cera") |
                        (datasetToClean[mainOrsecondProblem] == "Szara Cera"), mainOrsecondProblem] = "Szara cera"
    datasetToClean.loc[(datasetToClean[mainOrsecondProblem] == "widoczne naczynka") | 
                        (datasetToClean[mainOrsecondProblem] == "Widoczne Naczynka") |
                        (datasetToClean[mainOrsecondProblem] == "widoczne Naczynka"), mainOrsecondProblem] = "Widoczne naczynka"
    if mainOrsecondProblem == "Poboczny problem":
        datasetToClean.loc[(datasetToClean["Poboczny problem"] == "brak"), "Poboczny problem"] = "Brak"

def clearData(datasetToClean):
    datasetToClean.dropna()
    clearSkinTypeMisspelledData(datasetToClean)
    clearMainOrsecondProblemMisspelledData(datasetToClean, "Główny problem")
    clearMainOrsecondProblemMisspelledData(datasetToClean, "Poboczny problem")
    datasetToClean.drop_duplicates(inplace=True)
    datasetToClean = datasetToClean[datasetToClean["Wiek"] >= 16]

In [121]:
clearData(dataset)
dataset.head(3)

,ID,Typ cery,Wrażliwa,Główny problem,Poboczny problem,Wiek,Mycie,Serum na dzień,Krem na dzień,SPF,Serum na noc,Krem na noc,Punktowo,Maseczka,Peeling
0,1,Tłusta,0,Nadprodukcja sebum,Niedoskonałości,17,Neutrogena Visibly Clear Clear & Defend Facial...,Revox Just Niacinamide 10%,Tołpa - Authentic - Krem Matowa Skóra,SVR Sebiaclear Creme SPF 50,Eveline Cosmetics Beauty & Glow Matujące Serum...,SVR Sebiaclear Hydra,Bandi - Sebo Care - Pasta Punktowa na Niedosko...,Nacomi Next Lvl Kwas Salicylowy 2% + Nature Qu...,Brak
1,2,Mieszana,1,Nadprodukcja sebum,Niedoskonałości,20,Pharmaceris T Puri-Sebogel - antybakteryjny że...,Revox Just Niacinamide 10%,Bandi Medical Expert Anti Dry Emulsja silnie n...,Missha - All-Around Safe Block Aqua Sun Gel SP...,The Ordinary - Niacinamide 10% + Zinc 1% - Ser...,SVR Sebiaclear Hydra,Bandi - Sebo Care - Pasta Punktowa na Niedosko...,Nacomi Next Lvl Kwas Salicylowy 2% + Nacomi Bi...,BANDI ANTI ACNE antytrądzikowy peeling kwasowy...
2,3,Mieszana,1,Nadprodukcja sebum,Niedoskonałości,20,Pharmaceris T Puri-Sebogel - antybakteryjny że...,Revox Just Niacinamide 10%,Cetaphil - Pro Oil Control SPF30 - Krem Nawilż...,Missha - All-Around Safe Block Aqua Sun Gel SP...,The Ordinary - Niacinamide 10% + Zinc 1% - Ser...,Dermedic Melumin,Bandi - Sebo Care - Pasta Punktowa na Niedosko...,Nacomi Next Lvl Kwas Salicylowy 2% + Nacomi Bi...,BANDI ANTI ACNE antytrądzikowy peeling kwasowy...


In [6]:
encoder = None
encoders = {}
labelsDescription = {}
skinType = None
isSensitive = None
mainProblem = None
secondProblem = None
age = None
resultSkinCare = {}
models = []
accuracy = {}
products = {}
link = None
chosenProduct = None

askedColumnNames = ['Typ cery', 'Główny problem', 'Poboczny problem', 'Wrażliwa','Wiek']
categoricalColumnNames = ['Typ cery', 'Główny problem', 'Poboczny problem']
decisionColumnNames = ['Mycie','Serum na dzień','Krem na dzień','SPF','Serum na noc','Krem na noc','Punktowo','Maseczka','Peeling']
allColumns = askedColumnNames + decisionColumnNames
allCategoricalColumns = categoricalColumnNames + decisionColumnNames

In [7]:
def getProblemColumnIndex(problemName):
    match problemName:
        case 'Mycie':
            return 6
        case 'Serum na dzień':
            return 7
        case 'Krem na dzień':
            return  8
        case 'SPF' :
            return  9
        case 'Serum na noc':
            return  10
        case 'Krem na noc':
            return  11
        case 'Punktowo':
            return  12
        case 'Maseczka':
            return  13
        case 'Peeling':
            return  14
        case _:
            raise ValueError("Nie rozpoznano kategorii produktu.")

In [8]:
from sklearn import decomposition
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [23]:
import matplotlib.pyplot as plt

In [11]:
def createLabelEncoding(datasetToEncode):
    global encoders, labelsDescription
    
    for categoricalColumn in allCategoricalColumns:
        encoders[categoricalColumn] = LabelEncoder()
        uniqueValues = list(datasetToEncode[categoricalColumn].unique())
        encoders[categoricalColumn] = encoders[categoricalColumn].fit(uniqueValues)
        datasetToEncode[categoricalColumn] = encoders[categoricalColumn].transform(datasetToEncode[categoricalColumn])
        
    return datasetToEncode

In [12]:
labeledDataset = createLabelEncoding(dataset)

Typy danych w zbiorze po kodowaniu LabelEncoder 

In [13]:
labeledDataset.dtypes

ID                  int64
Typ cery            int32
Wrażliwa            int64
Główny problem      int32
Poboczny problem    int32
Wiek                int64
Mycie               int32
Serum na dzień      int32
Krem na dzień       int32
SPF                 int32
Serum na noc        int32
Krem na noc         int32
Punktowo            int32
Maseczka            int32
Peeling             int32
dtype: object

In [14]:
def defineModel(model):
    match model:
        case 'DecisionTreeClassifier':
            return DecisionTreeClassifier()
        case 'KNeighborsClassifier':
            return KNeighborsClassifier()
        case 'LogisticRegression':
            return LogisticRegression()
        case 'RandomForestClassifier':
            return RandomForestClassifier()

In [24]:
def makeSingleProblemModel(problemName, dumDf, dataset, modelName, maxDepth = None, criterion = None, maxFeatures = None, nEstimators = None, nn = None, C = None, penalty = None):
    problemIndex = 0
    global accuracy
    match problemName:
        case 'Mycie':
            problemIndex = 6
        case 'Serum na dzień':
            problemIndex = 7
        case 'Krem na dzień':
            problemIndex = 8
        case 'SPF' :
            problemIndex = 9
        case 'Serum na noc':
            problemIndex = 10
        case 'Krem na noc':
            problemIndex = 11
        case 'Punktowo':
            problemIndex = 12
        case 'Maseczka':
            problemIndex = 13
        case 'Peeling':
            problemIndex = 14
        case _:
            raise ValueError("Nie rozpoznano kategorii produktu.")

    X = dumDf.values[:, 1:6]
    yProblem = dataset.values[:, problemIndex]
    X_train, X_test, y_train, y_test = train_test_split(X, yProblem, test_size = 0.25, random_state = 100)
    model = defineModel(modelName)

    if criterion != None and maxDepth != None and maxFeatures == None and nEstimators == None:
        model.criterion = criterion
        model.max_depth = maxDepth

    if nn != None:
        model.n_neighbors = nn 

    if C != None and penalty != None:
        model.C = C
        model.penalty = penalty 
    
    if criterion != None and maxDepth != None and maxFeatures != None and nEstimators != None:
        model.criterion = criterion
        model.max_depth = maxDepth
        model.max_features = maxFeatures
        model.n_estimators = nEstimators
    
    model = model.fit(X_train, y_train)
    yPrediction = model.predict(X_test)
    accuracy[problemName] = str(accuracy_score(y_test, yPrediction)*100) # "{} - Accuracy : {}".format(problem_name,accuracy_score(y_test, y_pred)*100)
   
    return model

In [21]:
def tuneTreeModel(labeledData):
    for problem in decisionColumnNames:
        X = labeledData.values[:, 1:6]
        problemIndex = getProblemColumnIndex(problem)
        yProblem = dataset.values[:, problemIndex]    

        decisionTree = DecisionTreeClassifier()

        scaler = StandardScaler()
        pca = decomposition.PCA()
        pipe = Pipeline(steps=[('std_slc', scaler),
                                ('pca', pca),
                                ('dec_tree', decisionTree)])
        n_components = list(range(1,X.shape[1]+1,1))
        bestCriterion = ['gini', 'entropy']
        max_depth = [2,4,6,8,10,12]

        parameters = dict(pca__n_components=n_components,
                            dec_tree__criterion=bestCriterion,
                            dec_tree__max_depth=max_depth)
        clf_GS = GridSearchCV(pipe, parameters)
        clf_GS.fit(X, yProblem)

        bestCriterion = clf_GS.best_estimator_.get_params()['dec_tree__criterion']
        bestDepth = clf_GS.best_estimator_.get_params()['dec_tree__max_depth']

        problemModel = makeSingleProblemModel(problem, labeledData, dataset, maxDepth=bestDepth, criterion=bestCriterion, modelName='DecisionTreeClassifier')

In [16]:
def tuneKNN(labeledData):
    for problem in decisionColumnNames:
        X = labeledData.values[:, 1:6]
        problemIndex = getProblemColumnIndex(problem)
        yProblem = dataset.values[:, problemIndex]    

        knn = KNeighborsClassifier(n_neighbors=5)

        k_range = list(range(5, 31))
        parameters = dict(n_neighbors=k_range)
        grid = GridSearchCV(knn, parameters)
        grid.fit(X, yProblem)

        bestParam = grid.best_params_['n_neighbors']

        problemModel = makeSingleProblemModel(problem, labeledData, dataset, nn=bestParam, modelName='KNeighborsClassifier')

In [32]:
def tuneLogisticRegression(labeledData):
    for problem in decisionColumnNames:
        X = labeledData.values[:, 1:6]
        problemIndex = getProblemColumnIndex(problem)
        yProblem = dataset.values[:, problemIndex]    

        parameters = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
        logisiticRegression = LogisticRegression() 
        grid=GridSearchCV(logisiticRegression, parameters)
        grid.fit(X, yProblem)

        bestC = grid.best_params_['C']
        bestPenalty = grid.best_params_['penalty']

        problemModel = makeSingleProblemModel(problem, labeledData, dataset, C = bestC, penalty=bestPenalty, modelName='LogisticRegression')

In [44]:
def tuneRandomForest(labeledData):
    for problem in decisionColumnNames:
        X = labeledData.values[:, 1:6]
        problemIndex = getProblemColumnIndex(problem)
        yProblem = dataset.values[:, problemIndex] 


        param_grid = { 
        'n_estimators': [10, 100],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [4,5,6,7,8],
        'criterion' :['gini', 'entropy']
        }
        rfc = RandomForestClassifier()
        CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
        CV_rfc.fit(X, yProblem)
        CV_rfc.best_params_

        bestCriterion = CV_rfc.best_params_['criterion']
        bestMaxDepth = CV_rfc.best_params_['max_depth']
        bestMaxFeatures = CV_rfc.best_params_['max_features']
        bestNEstimators = CV_rfc.best_params_['n_estimators']


        print("criterion=" + str(bestCriterion))
        print("bestMaxDepth="+ str(bestMaxDepth))
        print("bestMaxFeatures="+ str(bestMaxDepth))
        print("bestNEstimators="+ str(bestNEstimators))

        problemModel = makeSingleProblemModel(problem, labeledData, dataset, 
                                                maxDepth=bestMaxDepth,
                                                criterion=bestCriterion,
                                                maxFeatures=bestMaxFeatures,
                                                nEstimators=bestNEstimators,
                                                modelName='RandomForestClassifier')

In [22]:
tuneTreeModel(labeledDataset)
accuracy

ValueError: X has 2 features, but DecisionTreeClassifier is expecting 5 features as input.

In [17]:
tuneKNN(labeledDataset)
accuracy

{'Mycie': '48.717948717948715',
 'Serum na dzień': '79.48717948717949',
 'Krem na dzień': '43.58974358974359',
 'SPF': '48.717948717948715',
 'Serum na noc': '23.076923076923077',
 'Krem na noc': '48.717948717948715',
 'Punktowo': '51.28205128205128',
 'Maseczka': '35.8974358974359',
 'Peeling': '66.66666666666666'}

In [36]:
tuneLogisticRegression(labeledDataset)
accuracy

C=100.0
penalty=l2
C=100.0
penalty=l2
C=1.0
penalty=l2
C=10.0
penalty=l2
C=0.01
penalty=l2
C=0.1
penalty=l2
C=10.0
penalty=l2
C=100.0
penalty=l2
C=1.0
penalty=l2


{'Mycie': '38.46153846153847',
 'Serum na dzień': '84.61538461538461',
 'Krem na dzień': '48.717948717948715',
 'SPF': '43.58974358974359',
 'Serum na noc': '25.64102564102564',
 'Krem na noc': '46.15384615384615',
 'Punktowo': '56.41025641025641',
 'Maseczka': '38.46153846153847',
 'Peeling': '53.84615384615385'}

In [45]:
tuneRandomForest(labeledDataset)
accuracy

criterion=gini
bestMaxDepth=8
bestMaxFeatures=8
bestNEstimators=10
criterion=gini
bestMaxDepth=5
bestMaxFeatures=5
bestNEstimators=100
criterion=entropy
bestMaxDepth=5
bestMaxFeatures=5
bestNEstimators=10
criterion=entropy
bestMaxDepth=5
bestMaxFeatures=5
bestNEstimators=10
criterion=gini
bestMaxDepth=8
bestMaxFeatures=8
bestNEstimators=100
criterion=entropy
bestMaxDepth=5
bestMaxFeatures=5
bestNEstimators=100
criterion=gini
bestMaxDepth=4
bestMaxFeatures=4
bestNEstimators=10
criterion=entropy
bestMaxDepth=5
bestMaxFeatures=5
bestNEstimators=100
criterion=gini
bestMaxDepth=8
bestMaxFeatures=8
bestNEstimators=10


{'Mycie': '66.66666666666666',
 'Serum na dzień': '92.3076923076923',
 'Krem na dzień': '58.97435897435898',
 'SPF': '64.1025641025641',
 'Serum na noc': '51.28205128205128',
 'Krem na noc': '56.41025641025641',
 'Punktowo': '53.84615384615385',
 'Maseczka': '56.41025641025641',
 'Peeling': '84.61538461538461'}